In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def get_weather(lon, lat):
    print("API 호출 중!!")

function = {
    "name": "create_quiz",
    "description": "정답과 질문을 만드는 퀴즈 생성기",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

llm = ChatOpenAI(temperature=0.1,).bind(
    function_call={
        "name" : "create_quiz" 
    },
    # function_call = auto 자유도를 줄 떄 사용 -> 모델이 우리의 함수를 사용 or Not
    functions=[function]
)

prompt = PromptTemplate.from_template("{theme} 에 대해서 얼마나 알고 있나요?")

chain = prompt | llm
response = chain.invoke({"theme" : "귀멸의 칼날"})

response = response.additional_kwargs["function_call"]["arguments"]
response

InvalidRequestError: Invalid value for 'function_call': no function named 'get_weather' was specified in the 'functions' parameter.